In [1]:
import pandas as pd
import numpy as np
import talib as ta
import os
import matplotlib.pyplot as plt
import warnings
import pymongo
from datetime import datetime
warnings.filterwarnings('ignore')
import pickle
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
dataCoin = load_obj('../../data/usdceth/symbolsData_2018032120211116_60min_v16_E')['60min']
dataCoin.columns= ['open','high','low','close','volume']
dataCoin

,open,high,low,close,volume
timestamp,,,,,
2018-03-21 12:00:00+00:00,580.19,581.44,570.00,573.58,4203.57962
2018-03-21 13:00:00+00:00,573.36,578.27,570.73,573.13,3038.19939
2018-03-21 14:00:00+00:00,573.12,578.50,571.57,575.50,3057.08944
2018-03-21 15:00:00+00:00,575.50,584.41,574.00,574.73,4885.99445
2018-03-21 16:00:00+00:00,575.00,577.48,556.02,556.62,11892.60047
...,...,...,...,...,...
2021-11-10 04:00:00+00:00,4691.48,4717.33,4677.91,4713.75,10383.83280
2021-11-10 05:00:00+00:00,4713.76,4716.44,4697.30,4701.07,7720.72120
2021-11-10 06:00:00+00:00,4701.08,4740.96,4697.28,4739.78,11182.21750


In [2]:
# dataCoin = pd.read_csv('../data/dataCoin.csv')
# dataCoin.set_index('datetime',inplace=True)
dataCoins = []
dataCoins.append(dataCoin.loc['2018-05-01':'2019-05-01'])
dataCoins.append(dataCoin.loc['2019-05-01':'2020-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-11-08']) #univ3上线一年前至今
dataCoins.append(dataCoin.loc['2021-05-01':'2021-11-08']) #univ3上线至今
dataCoins.append(dataCoin.loc[:]) #全样本

In [3]:
def cal_indicator(dataCoin):    
    dataCoin['pctChange'] = dataCoin['close'].pct_change()
    dataCoin['pctStd'] = ta.STDDEV(dataCoin['pctChange'], 24*5)*(24**0.5)
    dataCoin['pctStdVma'] = ta.SUM(dataCoin['pctStd']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
    dataCoin['closeVma'] = ta.SUM(dataCoin['close']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
#     dataCoin['pctStdVSma'] = ta.EMA(dataCoin.pctStdVma, 24*3)
#     dataCoin['pctStdVLma'] = ta.MA(dataCoin.pctStdVma, 24*10)
for data in dataCoins:
    cal_indicator(data)

In [4]:
def get_stat(df):
    se = df.describe()['pctStd']
    se = se.append(pd.Series(df.skew()['pctStd'],index=['skew']))
    se = se.append(pd.Series(df.kurt()['pctStd'],index=['kurt']))
    se = se.append(pd.Series(se['75%']-se['25%'],index=['75%-25%']))
    return se

In [5]:
ses = []
for data in dataCoins:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2019.5',
              '2019.5 - 2020.5',
              '2020.5 - 2021.5',
              '2020.5 - 2021.11', #univ3上线一年前至今
              '2021.5 - 2021.11', #univ3上线至今
              '2018.5 - 2021.11'] #全样本
df

,2018.5 - 2019.5,2019.5 - 2020.5,2020.5 - 2021.5,2020.5 - 2021.11,2021.5 - 2021.11,2018.5 - 2021.11
count,8664.000000,8688.000000,8664.000000,13248.000000,4488.000000,31797.000000
mean,0.044644,0.042845,0.043718,0.046962,0.053219,0.045382
std,0.019934,0.024069,0.019219,0.021299,0.023768,0.021612
min,0.012783,0.015876,0.012339,0.012339,0.024727,0.012339
25%,0.029081,0.029922,0.030795,0.033550,0.038463,0.031552
50%,0.040296,0.037440,0.038148,0.041200,0.046159,0.040367
75%,0.057182,0.048812,0.053368,0.055676,0.060077,0.054709
max,0.109565,0.202287,0.113070,0.162451,0.162451,0.202287
skew,0.773688,3.465241,1.196321,1.810818,2.339927,2.151403
kurt,0.220710,17.104674,1.194354,4.933070,6.288532,8.837176


In [6]:
data_over_ma = []
data_below_ma = []
dataCoinsMA = []
dataCoinsMA.append(dataCoin.loc['2018-05-01':'2021-05-01'])
dataCoinsMA.append(dataCoin.loc['2020-05-01':'2021-09-23'])
dataCoinsMA.append(dataCoin.loc['2020-12-01':'2021-04-30'])
for data in dataCoinsMA:
    cal_indicator(data)

for data in dataCoinsMA:
    data_over_ma.append(data[data['close'] >= data['closeVma']])
    data_below_ma.append(data[data['close'] < data['closeVma']])

In [7]:
ses = []
for data in data_over_ma:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2021.5',
              '2020.5 - 2021.9',
             '2020.12 - 2021.4']
print('OVER MA vol statistic')
df

OVER MA vol statistic


,2018.5 - 2021.5,2020.5 - 2021.9,2020.12 - 2021.4
count,14198.000000,8202.000000,2283.000000
mean,0.043394,0.046656,0.053131
std,0.018451,0.019248,0.017285
min,0.012853,0.012853,0.026479
25%,0.030926,0.033264,0.038098
50%,0.038511,0.041951,0.051236
75%,0.052187,0.055814,0.065966
max,0.113070,0.143626,0.093035
skew,1.219501,1.115917,0.519915
kurt,1.521242,1.220999,-0.865425


In [8]:
ses = []
for data in data_below_ma:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2021.5',
              '2020.5 - 2021.9',
              '2020.12 - 2021.4']
print('BELOW MA vol statistic')
df

BELOW MA vol statistic


,2018.5 - 2021.5,2020.5 - 2021.9,2020.12 - 2021.4
count,10931.000000,2863.000000,142.000000
mean,0.043431,0.054805,0.047444
std,0.024833,0.029000,0.015859
min,0.012339,0.012339,0.031590
25%,0.027975,0.037134,0.036468
50%,0.035857,0.046160,0.041695
75%,0.053459,0.063551,0.059884
max,0.202287,0.162451,0.090848
skew,2.639435,1.669879,1.320634
kurt,11.413552,3.072455,0.570902
